In [29]:
text = """
https://blog.csdn.net/made_in_china_too/article/details/79572451


https://blog.csdn.net/made_in_china_too/article/details/79547296


https://www.cnblogs.com/xxrxxr/p/7538430.html


https://blog.csdn.net/luoshixian099/article/details/51227754

https://blog.csdn.net/on2way/article/details/47730367?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-8.control&dist_request_id=1332023.6883.16189812533292951&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-8.control

https://blog.csdn.net/weixin_30794499/article/details/96787813?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control&dist_request_id=1332024.7935.16189846569031567&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control

https://blog.csdn.net/weixin_42398658/article/details/83271304?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-5.control&dist_request_id=1332023.6883.16189812533292951&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-5.control

"""
print(text)


https://blog.csdn.net/made_in_china_too/article/details/79572451


https://blog.csdn.net/made_in_china_too/article/details/79547296


https://www.cnblogs.com/xxrxxr/p/7538430.html


https://blog.csdn.net/luoshixian099/article/details/51227754

https://blog.csdn.net/on2way/article/details/47730367?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-8.control&dist_request_id=1332023.6883.16189812533292951&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-8.control

https://blog.csdn.net/weixin_30794499/article/details/96787813?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control&dist_request_id=1332024.7935.16189846569031567&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control

https://blog.csdn.net/weixin_42398658/article/details/83271304?utm_medium=distribute.pc_

In [4]:
from numpy import *
import matplotlib.pyplot as plt
import operator
import time

In [5]:
'''
    def loadDataSet(fileName):从数据集文件fileName中读入数据，返回数据列表dataMat和标签列表labelMat
'''
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('   ')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

'''
    def selectJrand(i,m):从[1,m)中，随机返回一个不等于i的数
'''
def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(random.uniform(0,m))
    return j

'''
    def clipAlpha(aj,H,L): 这个是剪辑函数，如果aj大于边界H，则返回H；如果aj小于边界L，则返回L；如果L<=aj<=H，则直接返回aj
'''
def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

'''
    def smoSimple(dataMatIn, classLabels, C, toler, maxIter):简化版的SMO算法。
    dataMatIn是数据列表，classLabels是标签列表，toler是松弛因子，C是toler的惩罚系数，maxIter是最大迭代次数。
    该函数首先遍历每个ai，然后再寻找当前ai对应合适的aj。如果遍历所有的ai都没有找到合适的aj，则重新遍历ai，但是该过程的最大次数为maxIter。
    该函数返回列表alphas、最佳的b
'''
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn);
    labelMat = mat(classLabels).transpose() #将classLabels从行向量转化为列向量
    b = 0; m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    iter = 0 #iter表示寻找ai-aj对的连续失败次数
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m): #这是外循环，用于遍历所有的参数ai
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
                 # multiply(alphas,labelMat).T表示列向量alphas与列向量labelMat对应元素相乘，然后再转置为行向量
            Ei = fXi - float(labelMat[i])
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                 # ‘if’检查是否违反KKT条件,如果没有违反则继续选择第二个参数aj
                j = selectJrand(i,m)  #随机从区间[i,m)中选出一个aj
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print ("L==H"); continue #重新选择ai
                eta = dataMatrix[i,:]*dataMatrix[i,:].T + dataMatrix[j,:]*dataMatrix[j,:].T - 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T
                if eta <= 0: print ("eta<=0"); continue #eta<=0不能符号条件，进行下一轮,重新选择ai
                alphas[j] += labelMat[j]*(Ei - Ej)/eta #计算第二个参数aj
                alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j]) #通过aj和ai的关系，由aj求出ai
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1 #ai与aj已经求出，alphaPairsChanged += 1
                print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
        if (alphaPairsChanged == 0): iter += 1 #如果遍历所有的ai都没有寻找到合适的aj，则表示寻找ai-aj对的连续失败次数iter+=1
        else: iter = 0 #找到了合适的ai-aj对，iter重置为0
        print ("iteration number: %d" % iter)
    return b,alphas


In [8]:
dataMatIn

[[3.542485, 1.977398],
 [3.018896, 2.556416],
 [7.55151, -1.58003],
 [2.114999, -0.004466],
 [8.127113, 1.274372],
 [7.108772, -0.986906],
 [8.610639, 2.046708],
 [2.326297, 0.265213],
 [3.634009, 1.730537],
 [0.341367, -0.894998],
 [3.125951, 0.293251],
 [2.123252, -0.783563],
 [0.887835, -2.797792],
 [7.139979, -2.329896],
 [1.696414, -1.212496],
 [8.117032, 0.623493],
 [8.497162, -0.266649],
 [4.658191, 3.507396],
 [8.197181, 1.545132],
 [1.208047, 0.2131],
 [1.928486, -0.32187],
 [2.175808, -0.014527],
 [7.886608, 0.461755],
 [3.223038, -0.552392],
 [3.628502, 2.190585],
 [7.40786, -0.121961],
 [7.286357, 0.251077],
 [2.301095, -0.533988],
 [-0.232542, -0.54769],
 [3.457096, -0.082216],
 [3.023938, -0.057392],
 [8.015003, 0.885325],
 [8.991748, 0.923154],
 [7.916831, -1.781735],
 [7.616862, -0.217958],
 [2.450939, 0.744967],
 [7.270337, -2.507834],
 [1.749721, -0.961902],
 [1.803111, -0.176349],
 [8.804461, 3.044301],
 [1.231257, -0.568573],
 [2.074915, 1.41055],
 [-0.743036, -1.73

In [33]:
dataMatIn,classLabels=loadDataSet('testSet.txt')
b,alphas=smoSimple(dataMatIn,classLabels,0.6,0.001,40)
print(b,alphas[alphas>0])

L==H
iter: 0 i:1, pairs changed 1
L==H
iter: 0 i:4, pairs changed 2
iter: 0 i:8, pairs changed 3
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
iteration number: 1
j not moving enough
iter: 1 i:4, pairs changed 1
j not moving enough
j not moving enough
L==H
L==H
L==H
L==H
j not moving enough
iter: 1 i:55, pairs changed 2
j not moving enough
L==H
j not moving enough
L==H
L==H
iteration number: 0
iter: 0 i:1, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
L==H


iteration number: 4
iter: 4 i:17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
iter: 5 i:29, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough


j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
iter: 7 i:52, pairs changed 1
j not moving enough
iter: 7 i:55, pairs changed 2
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough


iteration number: 7
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough

iter: 3 i:54, pairs changed 1
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iter: 3 i:52, pairs changed 1
j not moving enough
iter: 3 i:55, pairs changed 2
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not movi

iteration number: 5
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
iteration number: 11
j not moving enough
j not moving enough
j not moving enough
iteration number: 12
j not moving enough
j not moving enough
j not moving enough
iteration number: 13
j not moving enough
iter: 13 i:23, pairs changed 1
j not moving enough
L==H
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
L==H
j not moving enough
iteration number: 2
j not moving enough
j not moving en

iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
iter: 5 i:17, pairs changed 1
iter: 5 i:29, pairs changed 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
iter: 4 i:55, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
iter: 2 i:

iteration number: 8
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
iteration number: 11
j not moving enough
j not moving enough
j not moving enough
iteration number: 12
j not moving enough
j not moving enough
j not moving enough
iteration number: 13
j not moving enough
j not moving enough
j not moving enough
iteration number: 14
j not moving enough
j not moving enough
j not moving enough
iteration number: 15
j not moving enough
j not moving enough
iter: 15 i:55, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
iter: 4 i:54, pairs changed 1
iteration number: 0
j not moving 

iteration number: 1
j not moving enough
iteration number: 2
j not moving enough
iteration number: 3
j not moving enough
iteration number: 4
j not moving enough
iteration number: 5
j not moving enough
iteration number: 6
j not moving enough
iteration number: 7
j not moving enough
iteration number: 8
j not moving enough
iteration number: 9
j not moving enough
iteration number: 10
j not moving enough
iteration number: 11
j not moving enough
iteration number: 12
j not moving enough
iteration number: 13
j not moving enough
iteration number: 14
j not moving enough
iteration number: 15
j not moving enough
iteration number: 16
j not moving enough
iteration number: 17
j not moving enough
iteration number: 18
j not moving enough
iteration number: 19
j not moving enough
iteration number: 20
j not moving enough
iteration number: 21
j not moving enough
iteration number: 22
j not moving enough
iteration number: 23
j not moving enough
iteration number: 24
iter: 24 i:29, pairs changed 1
iteration numb

In [35]:
print(b,alphas[alphas>0])

[[-3.83765953]] [[0.1273107  0.24138304 0.36869374]]


In [36]:
alphas

matrix([[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.1273107 ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.24138304],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0

In [9]:
dataMatIn
classLabels
C = 0.6
toler = 0.001
maxIter = 40

In [11]:
dataMatrix = mat(dataMatIn)
dataMatrix

matrix([[ 3.542485e+00,  1.977398e+00],
        [ 3.018896e+00,  2.556416e+00],
        [ 7.551510e+00, -1.580030e+00],
        [ 2.114999e+00, -4.466000e-03],
        [ 8.127113e+00,  1.274372e+00],
        [ 7.108772e+00, -9.869060e-01],
        [ 8.610639e+00,  2.046708e+00],
        [ 2.326297e+00,  2.652130e-01],
        [ 3.634009e+00,  1.730537e+00],
        [ 3.413670e-01, -8.949980e-01],
        [ 3.125951e+00,  2.932510e-01],
        [ 2.123252e+00, -7.835630e-01],
        [ 8.878350e-01, -2.797792e+00],
        [ 7.139979e+00, -2.329896e+00],
        [ 1.696414e+00, -1.212496e+00],
        [ 8.117032e+00,  6.234930e-01],
        [ 8.497162e+00, -2.666490e-01],
        [ 4.658191e+00,  3.507396e+00],
        [ 8.197181e+00,  1.545132e+00],
        [ 1.208047e+00,  2.131000e-01],
        [ 1.928486e+00, -3.218700e-01],
        [ 2.175808e+00, -1.452700e-02],
        [ 7.886608e+00,  4.617550e-01],
        [ 3.223038e+00, -5.523920e-01],
        [ 3.628502e+00,  2.190585e+00],


In [13]:
labelMat = mat(classLabels).transpose() #将classLabels从行向量转化为列向量
# labelMat

In [14]:
b = 0; 
m,n = shape(dataMatrix)

In [16]:
alphas = mat(zeros((m,1)))
# alphas

In [17]:
iter = 0

In [18]:
alphaPairsChanged = 0
for i in range(m): #这是外循环，用于遍历所有的参数ai
    break
i

0

In [19]:
fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
    # multiply(alphas,labelMat).T表示列向量alphas与列向量labelMat对应元素相乘，然后再转置为行向量
fXi

0.0

In [23]:
multiply(alphas,labelMat).T

matrix([[-0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,
          0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
          0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
          0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
         -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
          0.,  0., -0., -0., -0., -0., -0., -0., -0.]])

In [22]:
dataMatrix*dataMatrix[i,:].T

matrix([[ 16.45930283],
        [ 15.74944568],
        [ 23.62676274],
        [  7.48352117],
        [ 31.31011654],
        [ 23.23121223],
        [ 34.5502158 ],
        [  8.76530388],
        [ 16.29538278],
        [ -0.56047978],
        [ 11.65350847],
        [  5.97217245],
        [ -2.38720614],
        [ 20.68613682],
        [  3.61193398],
        [ 29.98735792],
        [ 29.57379773],
        [ 23.43708958],
        [ 32.09373166],
        [  4.70087189],
        [  6.19516763],
        [  7.67904154],
        [ 28.85126395],
        [ 10.32526493],
        [ 17.18557231],
        [ 26.00106749],
        [ 26.30828953],
        [  7.09568772],
        [ -1.90677766],
        [ 12.08413697],
        [ 10.59876818],
        [ 30.14366779],
        [ 33.67857529],
        [ 24.52205584],
        [ 26.55162967],
        [ 10.1555109 ],
        [ 20.79607383],
        [  4.29629731],
        [  6.03878151],
        [ 37.20946573],
        [  3.23741434],
        [ 10.139

In [20]:
Ei = fXi - float(labelMat[i])
Ei

1.0

In [24]:
if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
    print(True)

True


In [25]:
toler

0.001

In [26]:
labelMat[i]*Ei

matrix([[-1.]])

In [ ]:
     
    if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
         # ‘if’检查是否违反KKT条件,如果没有违反则继续选择第二个参数aj
        j = selectJrand(i,m)  #随机从区间[i,m)中选出一个aj
        fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
        Ej = fXj - float(labelMat[j])
        alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
        if (labelMat[i] != labelMat[j]):
            L = max(0, alphas[j] - alphas[i])
            H = min(C, C + alphas[j] - alphas[i])
        else:
            L = max(0, alphas[j] + alphas[i] - C)
            H = min(C, alphas[j] + alphas[i])
        if L==H: print ("L==H"); continue #重新选择ai
        eta = dataMatrix[i,:]*dataMatrix[i,:].T + dataMatrix[j,:]*dataMatrix[j,:].T - 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T
        if eta <= 0: print ("eta<=0"); continue #eta<=0不能符号条件，进行下一轮,重新选择ai
        alphas[j] += labelMat[j]*(Ei - Ej)/eta #计算第二个参数aj
        alphas[j] = clipAlpha(alphas[j],H,L)
        if (abs(alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); continue
        alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j]) #通过aj和ai的关系，由aj求出ai
        b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
        b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
        if (0 < alphas[i]) and (C > alphas[i]): b = b1
        elif (0 < alphas[j]) and (C > alphas[j]): b = b2
        else: b = (b1 + b2)/2.0
        alphaPairsChanged += 1 #ai与aj已经求出，alphaPairsChanged += 1
        print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
if (alphaPairsChanged == 0): iter += 1 #如果遍历所有的ai都没有寻找到合适的aj，则表示寻找ai-aj对的连续失败次数iter+=1
else: iter = 0 #找到了合适的ai-aj对，iter重置为0
print ("iteration number: %d" % iter)

In [ ]:



iter = 0 #iter表示寻找ai-aj对的连续失败次数
while (iter < maxIter):
    alphaPairsChanged = 0
    for i in range(m): #这是外循环，用于遍历所有的参数ai
        fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
             # multiply(alphas,labelMat).T表示列向量alphas与列向量labelMat对应元素相乘，然后再转置为行向量
        Ei = fXi - float(labelMat[i])
        if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
             # ‘if’检查是否违反KKT条件,如果没有违反则继续选择第二个参数aj
            j = selectJrand(i,m)  #随机从区间[i,m)中选出一个aj
            fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
            Ej = fXj - float(labelMat[j])
            alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
            if (labelMat[i] != labelMat[j]):
                L = max(0, alphas[j] - alphas[i])
                H = min(C, C + alphas[j] - alphas[i])
            else:
                L = max(0, alphas[j] + alphas[i] - C)
                H = min(C, alphas[j] + alphas[i])
            if L==H: print ("L==H"); continue #重新选择ai
            eta = dataMatrix[i,:]*dataMatrix[i,:].T + dataMatrix[j,:]*dataMatrix[j,:].T - 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T
            if eta <= 0: print ("eta<=0"); continue #eta<=0不能符号条件，进行下一轮,重新选择ai
            alphas[j] += labelMat[j]*(Ei - Ej)/eta #计算第二个参数aj
            alphas[j] = clipAlpha(alphas[j],H,L)
            if (abs(alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); continue
            alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j]) #通过aj和ai的关系，由aj求出ai
            b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
            b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
            if (0 < alphas[i]) and (C > alphas[i]): b = b1
            elif (0 < alphas[j]) and (C > alphas[j]): b = b2
            else: b = (b1 + b2)/2.0
            alphaPairsChanged += 1 #ai与aj已经求出，alphaPairsChanged += 1
            print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
    if (alphaPairsChanged == 0): iter += 1 #如果遍历所有的ai都没有寻找到合适的aj，则表示寻找ai-aj对的连续失败次数iter+=1
    else: iter = 0 #找到了合适的ai-aj对，iter重置为0
    print ("iteration number: %d" % iter)